## Linking a dataset of real historical persons 

In this example, we deduplicate a more realistic dataset.  The data is based on historical persons scraped from wikidata.  Duplicate records are introduced with a variety of errors introduced.

Note, as explained in the [backends topic guide](https://moj-analytical-services.github.io/splink/topic_guides/backends.html#sqlite), SQLite does not natively support string fuzzy matching functions such as `damareau-levenshtein` and `jaro-winkler` (as used in this example). Instead, these have been imported as python User Defined Functions (UDFs). One drawback of python UDFs is that they are considerably slower than native-SQL comparisons. As such, if you are hitting issues with large run times, consider switching to DuckDB (or some other backend).

In [3]:
from splink.datasets import splink_datasets
from splink.sqlite.linker import SQLiteLinker
import altair as alt

import pandas as pd 
pd.options.display.max_rows = 1000
df = splink_datasets.historical_50k.sample(10000) # reduce size of dataset to reduce CI runtime

In [4]:
# Simple settings dictionary will be used for exploratory analysis
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name and l.surname = r.surname",
        "l.surname = r.surname and l.dob = r.dob",
        "l.first_name = r.first_name and l.dob = r.dob",
        "l.postcode_fake = r.postcode_fake and l.first_name = r.first_name",
    ],
}
linker = SQLiteLinker(df, settings)

linker.profile_columns(
    ["first_name", "postcode_fake", "substr(dob, 1,4)"], top_n=10, bottom_n=5
)

alt.VConcatChart(...)

In [5]:
linker.cumulative_num_comparisons_from_blocking_rules_chart()

alt.Chart(...)

In [6]:
import splink.sqlite.comparison_template_library as ctl
import splink.sqlite.comparison_library as cl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name and l.surname = r.surname",
        "l.surname = r.surname and l.dob = r.dob",
        "l.first_name = r.first_name and l.dob = r.dob",
        "l.postcode_fake = r.postcode_fake and l.first_name = r.first_name",
    ],
    "comparisons": [
        ctl.name_comparison("first_name", jaro_winkler_thresholds=[0.9], term_frequency_adjustments=True),
        ctl.name_comparison("surname", jaro_winkler_thresholds=[0.9], term_frequency_adjustments=True),
        cl.damerau_levenshtein_at_thresholds("dob", [1, 2], term_frequency_adjustments=True),
        cl.damerau_levenshtein_at_thresholds("postcode_fake", [1,2]),
        cl.exact_match("birth_place", term_frequency_adjustments=True),
        cl.exact_match("occupation",  term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01
}

linker = SQLiteLinker(df, settings)

In [7]:
linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.000134.
This means that amongst all possible pairwise record comparisons, one in 7,450.82 are expected to match.  With 49,995,000 total possible comparisons, we expect a total of around 6,710.00 matching pairs


In [8]:
linker.estimate_u_using_random_sampling(max_pairs=5e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).


In [9]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_names = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Iteration 1: Largest change in params was -0.526 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0367 in the m_probability of birth_place, level `Exact match`
Iteration 3: Largest change in params was -0.0136 in the m_probability of birth_place, level `All other comparisons`
Iteration 4: Largest change in params was -0.00542 in the m_probability of dob, level `All other comparisons`

EM converged after 4 iterations

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - s

In [10]:
training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.345 in the m_probability of first_name, level `Exact match first_name`
Iteration 2: Largest change in params was -0.0392 in the m_probability of first_name, level `Exact match first_name`
Iteration 3: Largest change in params was -0.00539 in the m_probability of surname, level `Exact match surname`

EM converged after 3 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


The final match weights can be viewed in the match weights chart:

In [11]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [12]:
linker.unlinkables_chart()

alt.LayerChart(...)

In [13]:
df_predict = linker.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_birth_place,bf_tf_adj_birth_place,occupation_l,occupation_r,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,match_key
0,12.016358,0.999759,Q18530059-11,Q18530059-14,jessie,jessie,3,0.001101,0.001101,44.03437,...,167.941885,1.406532,None,None,-1,NaN,None,1.0,1.0,0
1,12.016358,0.999759,Q18530059-11,Q18530059-5,jessie,jessie,3,0.001101,0.001101,44.03437,...,167.941885,1.406532,None,None,-1,NaN,None,1.0,1.0,0
2,-14.780044,0.000036,Q559411-1,Q8017419-2,william,william,3,0.057174,0.057174,44.03437,...,0.156087,1.000000,lexicographer,None,-1,0.000398,None,1.0,1.0,0
3,13.534864,0.999916,Q6222556-21,Q6222556-4,jean,jean,3,0.004606,0.004606,44.03437,...,0.156087,1.000000,None,None,-1,NaN,None,1.0,1.0,0
4,6.731764,0.990679,Q7527724-6,Q7527724-8,sir,sir,3,0.026735,0.026735,44.03437,...,1.000000,1.000000,politician,None,-1,0.088809,None,1.0,1.0,0


You can also view rows in this dataset as a waterfall chart as follows:

In [14]:
from splink.charts import waterfall_chart
records_to_plot = df_e.to_dict(orient="records")
linker.waterfall_chart(records_to_plot, filter_nulls=False)

alt.LayerChart(...)

In [15]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.95)

Completed iteration 1, root rows count 20
Completed iteration 2, root rows count 0


In [16]:
linker.cluster_studio_dashboard(df_predict, clusters, "dashboards/50k_cluster.html", sampling_method='by_cluster_size', overwrite=True)

from IPython.display import IFrame

IFrame(
    src="./dashboards/50k_cluster.html", width="100%", height=1200
)  

In [17]:
linker.roc_chart_from_labels_column("cluster",match_weight_round_to_nearest=0.02)

alt.Chart(...)

In [18]:
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.999,
    include_false_negatives=False,
    include_false_positives=True,
).as_record_dict()
linker.waterfall_chart(records)

alt.LayerChart(...)

In [19]:
# Some of the false negatives will be because they weren't detected by the blocking rules
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.5,
    include_false_negatives=True,
    include_false_positives=False,
).as_record_dict(limit=50)

linker.waterfall_chart(records)

alt.LayerChart(...)